05-01-2023 CR:
this notebook is based on the cc_order.ipynb file in the specsv1-xx file retrieved 19-12-2022. 
the notebook is being adapted to accomodate for importing the provided SelleckChem library file and exporting an iDot file.
the latter part of the notebook is based on codes by Andreina.

In [41]:
import json
import os.path
import time
import pandas as pd
import glob
import numpy as np
import csv 
from datetime import datetime
import re 


#
# Some good options for displaying dataframe in jupyter notebook
#

# settings to display more columns and rows
pd.set_option("max_colwidth", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 10) # None

print(os.getcwd())
os.chdir('../colo52-input/')
print(os.getcwd())

/share/data/analyses/christa/colopaint3D/colo52-input
/share/data/analyses/christa/colopaint3D/colo52-input


In [42]:
# Remove the zero before the well number
def strip_zeros(well):
    number = re.split('(\d+)', well)[1].lstrip('0')
    letter = re.split('(\d+)', well)[0]
    new_well = letter + number
    return new_well


# Remove extra white spaces that I entered by accident in plaid
def strip_spaces(a_str_with_spaces):
    # https://stackoverflow.com/questions/43332057/pandas-strip-white-space
    return a_str_with_spaces.replace(' ', '')

In [43]:
OutputDir = "idot-protocols" # Where do you want to save the idot protocols? 
if not os.path.exists(OutputDir):
    os.makedirs(OutputDir)


SupportDir = "support-files" # Where do you want to save the merged data csv? 
if not os.path.exists(SupportDir):
    os.makedirs(SupportDir)


ImportDir = "import-files"

In [44]:
## Split the protocol in water and dsmo parts

# solvent = 'dmso'
solvent = 'water'

In [45]:

# Add some idot settings
#

#Line 1
software = "1.7.2021.1019" # I-DOT Assay Studio software version
user_name = "pharmb_io" # User name/ID

#Line 2
sourceplate_type = "S.100 Plate" # Other options are "S.60 Plate" and "S.200 Plate"
max_volume_uL = 70 # allow for a bit of variation
max_volume_L = max_volume_uL / 1e6 # Source plate max volume (80000nL = 80uL= 8.0E-5 L) 
target_plate_type = "MWP 384" # Other options are "MWP 96" and "MWP 1536" 
waste_well = "Waste Tube" # Position of the waste well on the target carrier

#Line 3
dispense_to_waste = True # Enable/disable priming before dispensing (=True/False) 
dispense_to_waste_cycles = 2 # Number of priming cycles for each source well (=1/2/3)
dispense_to_waste_volume = 2e-8 # Dispensing volume for each priming cycle (=5e-8/.../1e-6) 
use_deionisation = True
optimization_level = "ReorderAndParallel" # Used protocol optimization process to reduce total dispensing time. Possible values are NoOptimization / Reorder / ReorderAndParalell
waste_error_handling_level = "Ask" # Checkpoint for the dispensing run if no droplets are detected during priming. Possible values are Ask / Abort / Continue 
save_liquids = "Ask" # Checkpoint for Liquid Library handling. Possible options are Ask / Never

### Import PLAID files, combine, and fill in missing values

In [46]:
#
# some conventions
#

# DMSO and water negative controls always in %
# Compound vol always in nL
# Well volume always in uL
# Stock conc always in mM

In [47]:
#
# Import all plaid files, do some cleaning, and combine them
#

# experiment code
exp_name = 'spher-colo52-'

# DMSO percentage that you allow in experiment
dmso_max_perc = 0.1

# H20 percentage that you allow in experiment (at times cmpds are dissolved in water)
h2o_max_perc = 5 # arbitrary, but I assume that water is not harmful like dmso. Also there is a control.

# Working volume in wells in ul
well_vol_uL = 40

plaid_folder = 'plaid_files'


# Spot multiple plates to one plate copy.
duplication_factor = 2.5 # For me specifically it means that I will take 25µl in the plate copy plate. 


# Combine all plaid-csv-files from specified folder into a dataframe, replace plateID "plate_1" in each file into plate_1 - plate_X
filelist = glob.glob(os.path.join(plaid_folder, '*.csv'))
df_combined = pd.DataFrame()
for idx, file in enumerate(sorted(filelist)):
    # read csv file and replace plateID plate_1 in all files to xxx_l1_p1
    df = pd.read_csv(file,index_col=False,converters={'cmpdnum': strip_spaces}, usecols=['plateID', 'well', 'cmpdname', 'CONCuM', 'cmpdnum'])
    # new_plate_name = 'plate_' + str(idx + 1)
    new_plate_name = os.path.splitext(os.path.basename(file))[0] # assign the barcode instead
    df['plateID'] = df['plateID'].str.replace('plate_1', new_plate_name)
    
    df_combined = pd.concat([df_combined, df], ignore_index=True)    

# drop all rows that have the '------' id
df_combined = df_combined[df_combined["plateID"].str.contains("----------") == False]


display(df_combined)


,plateID,well,cmpdname,CONCuM,cmpdnum
0,spher-colo52-P1-0,B02,colo-006,3.0,colo-006_3.
1,spher-colo52-P1-0,B03,colo-018,0.1,colo-018_0.1
2,spher-colo52-P1-0,B04,colo-009,3.0,colo-009_3.
3,spher-colo52-P1-0,B05,colo-012,10.0,colo-012_10
4,spher-colo52-P1-0,B06,colo-008,3.0,colo-008_3.
...,...,...,...,...,...
458,spher-colo52-P2-1,O09,colo-051,10.0,colo-051_10
459,spher-colo52-P2-1,O10,colo-041,0.3,colo-041_0.3
460,spher-colo52-P2-1,O11,dmso,0.1,dmso_0.1
461,spher-colo52-P2-1,O12,colo-045,10.0,colo-045_10


In [48]:
#
# Import information on the reference compounds and controls
#

df_controls_refcmpds = pd.read_csv("{}/colo52-controls-refcmpds.csv".format(SupportDir),index_col=False)
display(df_controls_refcmpds)

,cmpdname,max_stock,stock_unit,solvent,CONCuM,Plate Location
0,dmso,100.0,%,dmso,0.1,a12
1,water,100.0,%,water,0.5,b12
2,sorb,10.0,mM,dmso,10.0,c12
3,stau,0.1,mM,dmso,0.1,d12
4,etop,10.0,mM,dmso,2.5,e12
5,flup,10.0,mM,dmso,10.0,f12
6,fenb,10.0,mM,dmso,2.5,g12


In [49]:
# replace = dict(zip(["colo-004-0", "colo-004-1", "colo-004-2","colo-004-3" ,"colo-004-4"], ["colo-004"]*5))

# # start with a copy of last cells dataframe


# # replace matching part of a string with another string
# df_replaced['cmpdnum'] = df_replaced['cmpdnum'].replace(replace, regex=True)
# df_replaced['cmpdname'] = df_replaced['cmpdname'].replace(replace, regex=True)

# display(df_replaced)

In [50]:
## 
## Adjust layout to have the 2nd duplicated but in last rows 
##


df_copy = df_combined[df_combined['plateID'] == 'spher-colo52-P2-1'].copy()

# Split the well column into letter and number, and convert the number to integer
df_copy[['well_letter', 'well_number']] = df_copy['well'].str.extract(r'([A-Z])(\d{2})')

# Convert the well number to integer
df_copy['well_number'] = df_copy['well_number'].astype(int)

df_copy['old_well_number'] = df_copy['well_number']
df_copy['well_number'] = df_copy['well_number'] + 11

df_copy['old_well'] = df_copy['well']
df_copy['well'] = df_copy['well_letter'] + df_copy['well_number'].astype(str)

df_replaced = pd.concat([df_combined, df_copy[['plateID', 'well', 'cmpdname', 'CONCuM', 'cmpdnum']]], ignore_index=True)

display(df_replaced)


,plateID,well,cmpdname,CONCuM,cmpdnum
0,spher-colo52-P1-0,B02,colo-006,3.0,colo-006_3.
1,spher-colo52-P1-0,B03,colo-018,0.1,colo-018_0.1
2,spher-colo52-P1-0,B04,colo-009,3.0,colo-009_3.
3,spher-colo52-P1-0,B05,colo-012,10.0,colo-012_10
4,spher-colo52-P1-0,B06,colo-008,3.0,colo-008_3.
...,...,...,...,...,...
611,spher-colo52-P2-1,O20,colo-051,10.0,colo-051_10
612,spher-colo52-P2-1,O21,colo-041,0.3,colo-041_0.3
613,spher-colo52-P2-1,O22,dmso,0.1,dmso_0.1
614,spher-colo52-P2-1,O23,colo-045,10.0,colo-045_10


In [51]:
# Save the combined file
plaid_combined_filename = 'plates_combined'
df_replaced.to_csv("{}/{}.csv".format(SupportDir,plaid_combined_filename), index=False, sep=",")

### Load the SelleckChem library, assign compounds, and find optimal stocks

In [52]:
#
# Add well volumes
#

# add columns with default values
df_replaced['well_vol_uL'] = well_vol_uL

# adjust the well notations
df_replaced['well_w_zero'] = df_replaced['well']
df_replaced['well'] = df_replaced['well'].map(strip_zeros)


In [53]:
#
# Import library from SelleckChem
#

library_file = 'colo52-selleck'
df_selleck = pd.read_csv("{}/{}.csv".format(SupportDir,library_file),index_col=False)

display(df_selleck)

,code,Cat,Name,Plate Location,Rack Number,Formulation,Target,Pathway,Information,M.w.,CAS Number,DMSO (mg/mL)Max Solubility,DMSO (mM)Max Solubility,Water (mg/mL)Max Solubility,Water (mM)Max Solubility,URL,Formula,Form,Synonyms,SMILES,ALogP,HBA_Count,HBD_Count,RotatableBond
0,colo-001,S1002,ABT-737,a2,L2000-01,in 10mM DMSO,"Bcl-2,Autophagy",Apoptosis,"ABT-737 is a BH3 mimetic inhibitor of Bcl-xL, Bcl-2 and Bcl-w with EC50 of 78.7 nM, 30.3 nM and 197.8 nM in cell-free assays, respectively no inhibition observed against Mcl-1, Bcl-B or Bfl-1. Pha...",813.43,852808-04-9,100.00,122.94,<1,NaN,http://selleckchem.com/products/ABT-737.html,C42H45ClN6O5S2,free base,NaN,CN(C)CCC(CSC1=CC=CC=C1)NC2=CC=C(C=C2[N+]([O-])=O)[S](=O)(=O)NC(=O)C3=CC=C(C=C3)N4CCN(CC4)CC5=C(C=CC=C5)C6=CC=C(Cl)C=C6,8.198,3.0,2.0,16.0
1,colo-002,S1004,Veliparib (ABT-888),b2,L2000-01,in 10mM DMSO,PARP,DNA Damage,"Veliparib (ABT-888) is a potent inhibitor of PARP1 and PARP2 with Ki of 5.2 nM and 2.9 nM in cell-free assays, respectively. It is inactive to SIRT2. Phase 3.",244.29,912444-00-9,17.00,69.59,<1,NaN,http://selleckchem.com/products/ABT-888.html,C13H16N4O,free base,NSC 737664,CC1(CCCN1)C2=NC3=C(C=CC=C3[NH]2)C(N)=O,0.986,2.0,3.0,2.0
2,colo-003,S1011,Afatinib (BIBW2992),c2,L2000-01,in 10mM DMSO,"EGFR,HER2",Protein Tyrosine Kinase,"Afatinib (BIBW2992) inhibits EGFR/ErbB irreversibly in vitro with IC50 of 0.5, 0.4, 10, 14, 1 nM for EGFRwt, EGFR L858R , EGFR L858R/T790M ErbB2 (HER2) and ErbB4 (HER4), respectively.",485.94,439081-18-2,97.00,199.61,<1,NaN,http://selleckchem.com/products/BIBW2992.html,C24H25ClFN5O3,Free Base,NaN,CN(C)C/C=C/C(=O)NC1=CC2=C(NC3=CC=C(F)C(=C3)Cl)N=CN=C2C=C1OC4CCOC4,3.917,5.0,2.0,8.0
3,colo-004,S1013,Bortezomib (PS-341),d2,L2000-01,in 10mM DMSO,Proteasome,Proteases,Bortezomib (PS-341) is a potent 20S proteasome inhibitor with Ki of 0.6 nM. It exhibits favorable selectivity towards tumor cells over normal cells.,384.24,179324-69-7,76.00,197.79,<1,NaN,http://selleckchem.com/products/Bortezomib.html,C19H25BN4O4,Free Base,"LDP-341, MLM341",CC(C)CC(NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN=C2)B(O)O,2.183,4.0,2.0,9.0
4,colo-005,S1025,Gefitinib (ZD1839),e2,L2000-01,in 10mM DMSO,EGFR,Protein Tyrosine Kinase,"Gefitinib (ZD-1839) is an EGFR inhibitor for Tyr1173, Tyr992, Tyr1173 and Tyr992 in the NR6wtEGFR and NR6W cells with IC50 of 37 nM, 37nM, 26 nM and 57 nM, respectively.",446.90,184475-35-2,89.00,199.15,<1,NaN,http://selleckchem.com/products/Gefitinib.html,C22H24ClFN4O3,free base,NaN,COC1=C(OCCCN2CCOCC2)C=C3C(=NC=NC3=C1)NC4=CC=C(F)C(=C4)Cl,4.203,5.0,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,colo-048,S1116,Palbociclib (PD-0332991) HCl,a10,L2000-01,in 10mM Water,CDK,Cell Cycle,"Palbociclib (PD-0332991) HCl is a highly selective inhibitor of CDK4/6 with IC50 of 11 nM/16 nM in cell-free assays, respectively. It shows no activity against CDK1/2/5, EGFR, FGFR, PDGFR, InsR, e...",483.99,827022-32-2,3.00,6.20,30.00,61.98,http://selleckchem.com/products/PD-0332991.html,C24H30ClN7O2,Hydrochloride,NaN,Cl.CC(=O)C1=C(C)C2=CN=C(NC3=NC=C(C=C3)N4CCNCC4)N=C2N(C5CCCC5)C1=O,3.183,5.0,2.0,5.0
48,colo-049,S1224,Oxaliplatin,b10,L2000-01,in 2mM Water,DNA/RNA Synthesis,DNA Damage,"Oxaliplatin inhibits DNA synthesis by conforming DNA adducts in RT4, TCCSUP, A2780, HT-29, U-373MG, U-87MG, SK-MEL-2, and HT-144 cells.",397.29,61825-94-3,NaN,NaN,3.00,7.55,http://selleckchem.com/products/Eloxatin.html,C8H14N2O4Pt,free base,L-OHP,[Pt++].NC1CCCCC1N.[O-]C(=O)C([O-])=O,-3.423,0.0,2.0,1.0
49,colo-050,S7108,Encorafenib (LGX818),h7,L2000-01,in 10mM DMSO,Raf,MAPK,Encorafenib (LGX818) is a highly potent RAF inhibitor with selective anti-proliferative and apoptotic activity in cells expressing B-RAF(V600E) with EC50 of 4 nM. Phase 3.,540.01,1269440-17-6,100,185.18,NaN,NaN,https://www.selleckchem.com/products/lgx818.html,C22H27ClFN7O4S,NaN,NaN,CC(C)N1C=C(C(=N1)C2=C(C(=CC(=C2)Cl)NS(=O)(=O)C)F)C3=NC(=NC=C3)NCC(C)NC(=O)OC,NaN,N

In [54]:
#
# Assign the compound names
# 

df_library = df_selleck.copy()

# each compound was aribitrarily given a code: colo-001:colo-052 
# assign a compound to each code, and also prepare a collumn with a short name. 

reference = 'colo' # identifier for the compounds

# retrieve unique coded compounds from plaid 
ListOfCompounds = df_replaced['cmpdname'][df_replaced['cmpdname'].str.contains(reference)].unique()
ListOfCompounds.sort()

print(ListOfCompounds)

# check if the selleckchem library has the same number of compounds
print('Are the number of compounds the same? ' + (len(df_library) == len(ListOfCompounds)).__str__())

# assign the compounds 
df_library['cmpdname'] = ListOfCompounds

#display(df_selleck)

#
# retrieve information on solvent and stock concentration
# 

# translate Formulation to solvent and stock_conc
df_library['solvent'] = df_library['Formulation'].str.split(expand=True)[2]

temp = df_library['Formulation'].str.split(expand=True)[1]
df_library['max_stock'] = temp.str.split('(\d+)', expand=True)[1].astype('float') # stock concentration is in mM
df_library['stock_unit'] = temp.str.split('(\d+)', expand=True)[2] # keep this information
del temp

df_library['cmpdname_short'] = df_library['Name'][df_library['Name'].notnull()].str[:5]
df_library.loc[df_library['cmpdname_short'].isnull(), 'cmpdname_short']  = df_library.loc[df_library['cmpdname_short'].isnull(),'cmpdname'].str[:5]

#
# the selleck library only contains data on the compounds. Add the information on reference compounds and controls
#

df_library = pd.concat([df_library,df_controls_refcmpds],axis=0, ignore_index=True )

# Match the solvent names to names used in the PLAID tool
df_library.loc[df_library['solvent'] == 'DMSO', 'solvent'] = 'dmso'
df_library.loc[df_library['solvent'] == 'Water', 'solvent'] = 'water'

# prepare for stockfinder by adding solvent and max_stock collumns to the df_replaced
df_forstockfinder = df_replaced.merge(df_library[['max_stock', 'solvent', 'stock_unit', 'cmpdname']], 
    left_on = 'cmpdname',
    right_on = 'cmpdname',
    how = 'left',
    )


['colo-001' 'colo-002' 'colo-003' 'colo-004' 'colo-005' 'colo-006'
 'colo-007' 'colo-008' 'colo-009' 'colo-010' 'colo-011' 'colo-012'
 'colo-013' 'colo-014' 'colo-015' 'colo-016' 'colo-017' 'colo-018'
 'colo-019' 'colo-020' 'colo-021' 'colo-022' 'colo-023' 'colo-024'
 'colo-025' 'colo-026' 'colo-027' 'colo-028' 'colo-029' 'colo-030'
 'colo-031' 'colo-032' 'colo-033' 'colo-034' 'colo-035' 'colo-036'
 'colo-037' 'colo-038' 'colo-039' 'colo-040' 'colo-041' 'colo-042'
 'colo-043' 'colo-044' 'colo-045' 'colo-046' 'colo-047' 'colo-048'
 'colo-049' 'colo-050' 'colo-051' 'colo-052']
Are the number of compounds the same? True


In [55]:

# # A bit tricky to get the compound name properly. 
# df_controls_refcmpds.rename(columns={'cmpdname': 'Name'}, inplace=True)

# df_library = pd.concat([df_library,df_controls_refcmpds],axis=0, ignore_index=True )

# # Match the solvent names to names used in the PLAID tool
# df_library.loc[df_library['solvent'] == 'DMSO', 'solvent'] = 'dmso'
# df_library.loc[df_library['solvent'] == 'Water', 'solvent'] = 'water'

# df_library['cmpdcode'] = df_library['cmpdname'] 
# df_library['cmpdname'] = df_library['Name'].str[:6]

In [56]:
#
# Calculate stock conc v2 - idot from selleck  
#

# Stock conc function
def stockfinder_idot(x_uM,max_stock,solvent, stock_unit):
    
    #  ----------------------------------------------------------------------------------------------------------------------------
    #     #  DESCRIPTION
    #     #  v1 * c1 = v2 * c2
    #     #  volume stock solution (nl) * concentration stock solution (mM)  = volume destination (ul) * concentration final (uM)
    #     #
    #     #  c1 = TO BE CALCULATED
    #     #  v1 = volume of stock in ul (minimum is 8 and maximum is dependent on dmso limits)
    #     #  c2 = target concentration in uM "CONCuM" value
    #     #  v2 = working_volume_ul (e.g. 40 ul)
    #     #
    #     #  c1_max (highest stock concentration that can be used for target concentration)
    #     #  c1_min (lowest stock concentration that can be used for target concentration)
    #  ----------------------------------------------------------------------------------------------------------------------------

    # check if 'dmso' or 'water' (notation is SelleckChem's)
    max_perc = []
    if solvent == 'dmso':
        max_perc = dmso_max_perc
    elif solvent == 'water':
        max_perc = h2o_max_perc
    else:
        max_perc = 0.001 # if there is an unknown solvent, put the stock_conc to 0 

    # set available stock concentrations
    availstocks_mM = list([max_stock, 1.0, 0.1, 0.01, 0.001, 0.0001]) # think about whether it really needs to be a list # sort them from highest to lowest for later

    # minimum volume to dispense
    minV1_nl = 8
     # maximum volume to be dispensed while staying below dmso limits            
    maxV1_nl = (max_perc / 100) * (well_vol_uL*1000)                                              

    if stock_unit == 'mM':
        # calculate  lowest and highest stock that can be used
        c1_low = (well_vol_uL * x_uM) / maxV1_nl                                    
        c1_high = (well_vol_uL * x_uM) / minV1_nl                                  
    elif stock_unit == '%':
        # calculate  lowest and highest stock that can be used
        c1_low = 100  # always the lowest available stock %                            
        c1_high = 100 # in % always the highest available stock %
        #TODO: something is weird here, think about it clearly
    else: 
        print('unknown stock unit')

     # find possible stock concentrations from available
    possible_stocks = [i for i in availstocks_mM if i >= c1_low and i <= c1_high] 

    # select highest availbale stock from list
    if not possible_stocks:
        return 0                 
    else:
        highest_stock = max(possible_stocks)                                          
        return [highest_stock, availstocks_mM]


#
#
# Add new column with result from applying stock_concentration calculation function
#
#

df_with_stock = df_forstockfinder.copy()
df_with_stock['CONCuM'] = df_with_stock['CONCuM'].astype('float')
df_with_stock['max_stock'] = df_with_stock['max_stock'].astype('float')

# Apply the stockfinder function using information from three columns
df_with_stock[['stock_conc_mM', 'availstocks_mM']] =  df_with_stock.apply(lambda x: stockfinder_idot(x.CONCuM, x.max_stock, x.solvent, x.stock_unit), axis=1, result_type='expand')
df_with_stock = df_with_stock.drop(columns='max_stock')
df_with_stock['cmpd_w_stock'] = df_with_stock['cmpdname'] + "[" + df_with_stock['stock_conc_mM'].astype(str)+ "]"


#
# Print Warning and Display all rows with zero values
#
df_stock_conc_zero = df_with_stock[df_with_stock['stock_conc_mM'] == 0]
if not df_stock_conc_zero.empty:
    print("Warning - following rows could not be assigned a Stock Concentration")
    display(df_stock_conc_zero)

In [57]:
#
# Set some unassigned stock concentrations
# 

# for etop, fenb, flup, set the stock concentration to 30
df_with_stock.loc[df_with_stock['cmpdname'].isin(['flup', 'fenb','etop']), ['stock_conc_mM', 'availstocks_mM']] = 10

# Adjust the compound with stock name as well 
df_with_stock['cmpd_w_stock'] = df_with_stock['cmpdname'] + "[" + df_with_stock['stock_conc_mM'].astype(str)+ "]"



df_stock_conc_zero = df_with_stock[df_with_stock['stock_conc_mM'] == 0]
if not df_stock_conc_zero.empty:
    print("Warning - following rows could not be assigned a Stock Concentration")
    display(df_stock_conc_zero)


### Run some tests and calculate backfills

In [58]:
#
# Calculate compound volumes
#

# Copy last cells df
df_w_cmpd = df_with_stock.copy()

# Convert column types
df_w_cmpd['CONCuM'] = df_w_cmpd['CONCuM'].astype(float)
df_w_cmpd['well_vol_uL'] = df_w_cmpd['well_vol_uL'].astype(float)
df_w_cmpd['stock_conc_mM'] = df_w_cmpd['stock_conc_mM'].astype(float)

# Calculate compound/dmso volumes
# We split df in two because DMSO is calculated in % other in uM
# dmso, water, and water+ concentration in field CONCuM is % and not uM

# compounds 
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == 'mM', 'CompVol_nL'] = (df_w_cmpd['CONCuM'] * df_w_cmpd['well_vol_uL']) / df_w_cmpd['stock_conc_mM']

# dmso / water conc is in % even although column name is CONCuM
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == '%', 'CompVol_nL'] = (df_w_cmpd['well_vol_uL'] * 1000) * (df_w_cmpd['CONCuM'] / 100)

# prepare a column fo the idot
df_w_cmpd['CompVol_nL'] = df_w_cmpd['CompVol_nL'].round() # not sure if I can do this
df_w_cmpd['CompVol_uL'] = df_w_cmpd['CompVol_nL'] / 1000 # Transfer Volume is in µl
# df_w_cmpd['']


#Display all rows
# with pd.option_context('display.max_rows', None,):



In [59]:
# More statistics

number_of_replicates = df_w_cmpd.groupby(['cmpdname','stock_conc_mM'])['cmpdname'].count().reset_index(name ='replicates').sort_values(['stock_conc_mM'], ascending=[False])
print("Number of replicates per_compound_and_stock")
with pd.option_context('display.max_rows', 10, 'display.min_rows', 30):
        display(number_of_replicates)

Number of replicates per_compound_and_stock


,cmpdname,stock_conc_mM,replicates
109,water,100.00,8
104,dmso,100.00,40
45,colo-023,10.00,4
31,colo-016,10.00,4
91,colo-046,10.00,8
...,...,...,...
54,colo-028,0.10,4
0,colo-001,0.10,4
34,colo-018,0.01,4
6,colo-004,0.01,4


In [60]:
# 
# Calculate backfill
#

if 'backfill_to_vol_uL' in locals():
    del backfill_to_vol_uL
else:
    print('')

# User defined volume in nL to backfill to (if this is set it will override calculated value)
# Comment out value if not used
# backfill_to_vol_uL = 0.04

# df_backfill = df_w_cmpd.copy()

# calculate max DMSO that is allowed in well (before toxic, calculated from well_vol_uL and dmso_max_perc)
maxDMSO = (dmso_max_perc / 100) * (well_vol_uL) # in ul --> also use this value for water


if 'backfill_to_vol_uL' in locals():
    target_vol_DMSO_per_well = backfill_to_vol_uL
    print("User defined max volume to backfill to: " + str(backfill_to_vol_uL))
else:
    target_vol_DMSO_per_well = maxDMSO
    print("Max allowed DMSO in well (before toxic, calculated from well_vol_uL and dmso_max_perc) (nL): " + str(maxDMSO))
print('')    


df_w_cmpd['backfill_vol_uL'] = target_vol_DMSO_per_well - df_w_cmpd['CompVol_uL']

# Warn if backfill is negative
df_backfill_negative = df_w_cmpd[df_w_cmpd.backfill_vol_uL < 0]
if(df_backfill_negative.empty):
    print("OK, no negative backfill found")
else:
     print("WARNING! Negative backfill results found")
     print("They will be set to 0")
     display(df_backfill_negative)

# Set backfill to 0 if negative  
df_w_cmpd.loc[df_w_cmpd['backfill_vol_uL'] < 0, 'backfill_vol_uL'] = 0


# Add a column with solvent + stock (this bit will only work once)
df_w_cmpd = df_w_cmpd.merge(df_controls_refcmpds[['max_stock', 'cmpdname']], left_on='solvent', right_on='cmpdname').drop('cmpdname_y', axis=1)
df_w_cmpd.rename(columns = {'cmpdname_x':'cmpdname', 'max_stock':'solvent_stock'}, inplace = True)
df_w_cmpd['solvent_w_stock'] = df_w_cmpd['solvent'] + "[" + (df_w_cmpd['solvent_stock']).astype(str) + "]"


with pd.option_context('display.max_rows', 20, 'display.min_rows', 20):
        display(df_w_cmpd)


Max allowed DMSO in well (before toxic, calculated from well_vol_uL and dmso_max_perc) (nL): 0.04

WARNING! Negative backfill results found
They will be set to 0


,plateID,well,cmpdname,CONCuM,cmpdnum,well_vol_uL,well_w_zero,solvent,stock_unit,stock_conc_mM,availstocks_mM,cmpd_w_stock,CompVol_nL,CompVol_uL,backfill_vol_uL
313,spher-colo52-P2-1,B7,colo-049,10.0,colo-049_10,40.0,B07,water,mM,2.0,"[2.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-049[2.0],200.0,0.20,-0.16
323,spher-colo52-P2-1,C7,water,0.5,water_0.5,40.0,C07,water,%,100.0,"[100.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",water[100.0],200.0,0.20,-0.16
340,spher-colo52-P2-1,F11,water,0.5,water_0.5,40.0,F11,water,%,100.0,"[100.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",water[100.0],200.0,0.20,-0.16
352,spher-colo52-P2-1,F2,colo-049,3.0,colo-049_3.,40.0,F02,water,mM,2.0,"[2.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-049[2.0],60.0,0.06,-0.02
371,spher-colo52-P2-1,G10,colo-049,3.0,colo-049_3.,40.0,G10,water,mM,2.0,"[2.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-049[2.0],60.0,0.06,-0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,spher-colo52-P2-1,F13,colo-049,3.0,colo-049_3.,40.0,F13,water,mM,2.0,"[2.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-049[2.0],60.0,0.06,-0.02
525,spher-colo52-P2-1,G21,colo-049,3.0,colo-049_3.,40.0,G21,water,mM,2.0,"[2.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-049[2.0],60.0,0.06,-0.02
583,spher-colo52-P2-1,M13,water,0.5,water_0.5,40.0,M13,water,%,100.0,"[100.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",water[100.0],200.0,0.20,-0.16
600,spher-colo52-P2-1,N19,water,0.5,water_0.5,40.0,N19,water,%,100.0,"[100.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",water[100.0],200.0,0.20,-0.16


,plateID,well,cmpdname,CONCuM,cmpdnum,well_vol_uL,well_w_zero,solvent,stock_unit,stock_conc_mM,availstocks_mM,cmpd_w_stock,CompVol_nL,CompVol_uL,backfill_vol_uL,solvent_stock,solvent_w_stock
0,spher-colo52-P1-0,B2,colo-006,3.0,colo-006_3.,40.0,B02,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-006[10.0],12.0,0.012,0.028,100.0,dmso[100.0]
1,spher-colo52-P1-0,B3,colo-018,0.1,colo-018_0.1,40.0,B03,dmso,mM,0.1,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-018[0.1],40.0,0.040,0.000,100.0,dmso[100.0]
2,spher-colo52-P1-0,B4,colo-009,3.0,colo-009_3.,40.0,B04,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-009[10.0],12.0,0.012,0.028,100.0,dmso[100.0]
3,spher-colo52-P1-0,B5,colo-012,10.0,colo-012_10,40.0,B05,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-012[10.0],40.0,0.040,0.000,100.0,dmso[100.0]
4,spher-colo52-P1-0,B6,colo-008,3.0,colo-008_3.,40.0,B06,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-008[10.0],12.0,0.012,0.028,100.0,dmso[100.0]
5,spher-colo52-P1-0,B7,colo-025,10.0,colo-025_10,40.0,B07,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-025[10.0],40.0,0.040,0.000,100.0,dmso[100.0]
6,spher-colo52-P1-0,B8,sorb,10.0,sorb_10,40.0,B08,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",sorb[10.0],40.0,0.040,0.000,100.0,dmso[100.0]
7,spher-colo52-P1-0,B9,colo-015,0.3,colo-015_0.3,40.0,B09,dmso,mM,1.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-015[1.0],12.0,0.012,0.028,100.0,dmso[100.0]
8,spher-colo52-P1-0,B10,colo-027,10.0,colo-027_10,40.0,B10,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-027[10.0],40.0,0.040,0.000,100.0,dmso[100.0]
9,spher-colo52-P1-0,B11,colo-032,10.0,colo-032_10,40.0,B11,dmso,mM,10.0,"[10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]",colo-032[10.0],40.0,0.040,0.000,100.0,dmso[100.0]


### Load previous source layouts

In [61]:
## Import the source layouts from the tp53 project

project = 'tp53colo53-'

# name = project + 'idot_cmpds'
# idot_df = pd.read_csv("{}/{}_{}.csv".format(SupportDir,name,solvent), sep=',')

# name = project + 'idot_source'
# idot_df_tosource = pd.read_csv("{}/{}_{}.csv".format(ImportDir,name, solvent), sep=',')

name = project + 'source'
df_source = pd.read_csv("{}/{}_{}.csv".format(ImportDir,name, solvent), sep=',')

### generate source plate layouts

the first source plates is filled with compounds at the highest stock concentrations, then every following source plate will be a copy of this plate as long as there are lower stock concentrations. 

since this experiment takes a SelleckChem library, the first source plate layout takes on their layout

In [62]:
# df_library_per_solvent = df_library[df_library['solvent'] == solvent]
# df_w_cmpd_per_solvent = df_w_cmpd[df_w_cmpd['solvent'] == solvent]

In [63]:
# #
# # Collect the information for the source plates first in the library dataframe, and then in a dictionary
# #

# # for some columns convert the selleckchem format into a format that the idot can read
# temp = df_library_per_solvent['Plate Location'].str.split('(\d+)', expand=True)
# # df_library['well'] = temp[0].str.upper() + temp[1].str.pad(2,fillchar='0') # does not work with the idot
# df_library_per_solvent = df_library_per_solvent.copy()
# df_library_per_solvent['well'] = temp[0].str.upper() + temp[1]
# del temp

# # set up a dictionary to capture source well information
# source_dict = {}

# source_dict['well'] = df_library_per_solvent.set_index('cmpdname')['well'].to_dict()
# source_dict['stock_unit'] = df_library_per_solvent.set_index('cmpdname')['stock_unit'].to_dict()
# source_dict['availstocks_mM'] = df_w_cmpd_per_solvent[['stock_conc_mM','cmpdname']].groupby('cmpdname').apply(lambda x: list(np.unique(x))).apply(lambda x: x[::-1]).to_dict()

In [64]:
# #
# # Dynamically generate source plates as long as there are lower stock concentrations
# #

# # This code puts all similar stock concentrations together on a plate

# iConc = 0
# count = 0

# # prepare an empty dataframe
# df_source = pd.DataFrame(columns=['sourceID', 'well', 'Compound', 'CONCmM'])


# while count < len(source_dict['availstocks_mM']):

#         count = 0
#         keys = []
#         vals = []
#         cmpds = []

#         # list all compounds at a given stock concentration
#         for key, val in source_dict['availstocks_mM'].items():
            
#             flag = True
#             try:
#                 val[iConc]
#             except IndexError:
#                 flag = False

#             if flag:
#                 vals.append(val[iConc]) #
#                 keys.append(key)
#                 cmpds.append(key + "[" + str(val[iConc]) + "]")
#             else: 
#                 count = count + 1

#         # Keep on printing source plates until the there are no smaller stock concentrations, otherwise report the number of source plates
#         if count < len(source_dict['availstocks_mM']):

#             df_temp = pd.DataFrame({'sourceID': 'source_' + str(iConc+1),
#                             'well': [source_dict['well'][x] for x in keys],
#                             'Compound': cmpds,
#                             'CONCmM': vals, # this is actually mM!
#                             })

#             df_source = pd.concat([df_source, df_temp],axis=0, ignore_index=True )

#         else: 
#             print('There are a total of ' + str(iConc) + ' source plates!')

        
#         iConc = iConc + 1


In [65]:
# # Make it simpler for water compounds

# if solvent == "water":
#     # Assign these new wells
#     df_source['well'] = ["A1", "A2", "B1", "A3", "A4", "A5", "A6"]

#     # Assign a single source 
#     df_source['sourceID'] = 'source_1'

In [66]:
# 
# Merge source and compounds
#

temp = df_w_cmpd.merge(df_source[['sourceID', 'well', 'Compound']], left_on='cmpd_w_stock', right_on='Compound', suffixes=('_plaid','_source')).drop('Compound', axis=1)

# add a column with the backfill source 
idot_df = temp.merge(df_source[['well', 'Compound']], left_on='solvent_w_stock', right_on='Compound', suffixes=("",'')).drop('Compound', axis=1)
idot_df.rename(columns = {'well': 'well_backfill'}, inplace = True)


In [67]:
# Assign the source well to aliquot to 

source_well_vol_uL = 5 # minimum volume per source well in µl

sources = np.sort(idot_df['sourceID'].unique()).tolist()
sources.append(float('nan'))


idot_df['to_sourceID'] = np.nan


for i in range(len(sources)-1): 
    idot_df.loc[idot_df['sourceID'] == sources[i],'to_sourceID'] = sources[i+1] # Do a merge operation here!


# prepare the dataframe for printing
idot_df = pd.merge(idot_df,df_source[df_source['sourceID'] != sources[0]], left_on=['to_sourceID', 'well_source'], right_on=['sourceID', 'well'], suffixes=("", "_dest"), how='outer')
idot_df['to_source_vol_uL'] = (idot_df['CONCmM'] * source_well_vol_uL) / idot_df['stock_conc_mM']
idot_df['to_source_backfill_vol_uL'] = source_well_vol_uL - idot_df['to_source_vol_uL']


In [68]:
# Prepare a dataframe for creating the experimental plates

idot_df['X'] = ''
idot_df['Y'] = ''
idot_df['New Array'] = ''

In [69]:
# # Prepare a dataframe for creating the source plates

# idot_df_tosource = idot_df.copy()
# idot_df_tosource = idot_df_tosource.drop(['well_plaid', 'well_w_zero', 'CompVol_nL', 'CompVol_uL', 'availstocks_mM', 'plateID', 'backfill_vol_uL', 'cmpdnum', 'CONCuM'], axis=1)
# idot_df_tosource = idot_df_tosource.drop_duplicates()
# idot_df_tosource = idot_df_tosource[idot_df_tosource['to_source_vol_uL'].notnull()]

### Do some statistics

In [70]:

#
# calculate some statistics on the compound protocols
#


volume_per_compound_and_stock2 = idot_df.groupby(['solvent_w_stock','sourceID'])['backfill_vol_uL'].agg('sum').reset_index(name ='TotalVol_uL').sort_values(['sourceID'], ascending=[False])
with pd.option_context('display.max_rows', None, 'display.min_rows', 30):
    display(volume_per_compound_and_stock2)


volume_per_compound_and_stock = idot_df.groupby(['cmpd_w_stock','sourceID'])['CompVol_uL'].agg('sum').reset_index(name ='TotalVol_uL').sort_values(['sourceID'], ascending=[False])
with pd.option_context('display.max_rows', None, 'display.min_rows', 30):
    display(volume_per_compound_and_stock)

,solvent_w_stock,sourceID,TotalVol_uL
0,water[100.0],source_1,0.416


,cmpd_w_stock,sourceID,TotalVol_uL
0,colo-048[1.0],source_1,0.208
1,colo-048[10.0],source_1,0.208
2,colo-049[1.0],source_1,0.048
3,colo-049[2.0],source_1,1.120
4,water[100.0],source_1,1.600


In [71]:
# #
# # calculate some statistics on the source protocols 
# #

# volume_per_compound_and_stock2 = idot_df_tosource.groupby(['solvent_w_stock','sourceID'])['to_source_backfill_vol_uL'].agg('sum').reset_index(name ='TotalVol_uL').sort_values(['sourceID'], ascending=[False])
# with pd.option_context('display.max_rows', None, 'display.min_rows', 30):
#     display(volume_per_compound_and_stock2)

# volume_per_compound_and_stock = idot_df_tosource.groupby(['cmpd_w_stock','sourceID'])['to_source_vol_uL'].agg('sum').reset_index(name ='TotalVol_uL').sort_values(['sourceID'], ascending=[False])
# with pd.option_context('display.max_rows', 15, 'display.min_rows', 30):
#     display(volume_per_compound_and_stock)


In [72]:
# # from Jonne #@title Generate DMSO source plate

# def createplate(size, direction):

#     import string

#     if size == 96:
#         colr = 13
#         rowr = 8
    
#     if size == 384:
#         colr = 25
#         rowr = 16
    

#     row = list(string.ascii_uppercase[:rowr])
#     col = [(f'{i:02d}') for i in range(1, colr, 1)]
#     wells = []
      
#     if direction == "vert":
#         for c in col:
#             for r in row:
#                 wells.append(str(r+c))
#         return(wells)
    
#     else:
#         for r in row:
#             for c in col:
#                 wells.append(str(r+c))
#         return(wells)

# def assign_DMSOsource(max_volume, dfvolumes):
    
    
#     wellcapacity = int(max_volume *1e06) # wellcapacity based on idot plate
#     well_state = {"well_number": 0, "current_amount": wellcapacity}
#     #dfvolumes = dfvolumes.to_list()
#     DMSOwells = createplate(size=96,direction="vert")
    
#     sourcewelllist = []

#     for volume in dfvolumes:
#       #print(volume)
#         remaining = well_state["current_amount"] - volume
#         if remaining < 0:
#             well_state["well_number"] += 1
#             well_state["current_amount"] = wellcapacity
#             wellindex = well_state["well_number"]
#             sourcewelllist.append(DMSOwells[wellindex])
#         else:
#             well_state["current_amount"] -= volume
#             wellindex = well_state["well_number"]
#             sourcewelllist.append(DMSOwells[wellindex])
            
#     sourcewells = [*set(sourcewelllist)]      
#     return sourcewelllist

# # dfvolumes = dfDMSO["Volume [uL]"].to_list()
# dfvolumes = idot_df_tosource['to_source_backfill_vol_uL'].to_list()

# idot_df_tosource['to_source_backfill_well'] =  assign_DMSOsource(max_volume_L, dfvolumes)
# idot_df_tosource['to_source_backfill_well'] = idot_df_tosource['to_source_backfill_well'].map(strip_zeros)
# # dfDMSO[['Liquid Name','Source Plate',]] = 'DMSO', 'DMSOsource'
# # dmsoSOURCE = dfDMSO[["Liquid Name","Source Plate","Source Well"]].drop_duplicates()

In [73]:
## Duplicate the volumes to account for multiple plates


# Increase the volumes to adjust for multiple plates
idot_df[['CompVol_uL', 'backfill_vol_uL', 'CompVol_nL']] =  idot_df[['CompVol_uL', 'backfill_vol_uL', 'CompVol_nL']] * duplication_factor

# Drop rows from the idot_df when plateID is NaN (water or dmso compounds are skipped respectively)
idot_df = idot_df[idot_df['plateID'].notnull()]


### Print the idot protocols using an adapted version of Andreina's conversion code

##### dmso compounds

In [74]:
#
# Print the protocols where the experimental plates are spotted
#

In [75]:
# Source to plate protocols

# print the plates here:
for source in np.sort(idot_df['sourceID'].unique()).tolist(): 

    protocol_name = "IDOT-" + exp_name + "source" + source[-1] + "ONcmpd" + "_" + solvent
    idot_filename = "{}/{}.csv".format(OutputDir,protocol_name)
    sourceplate_name = "source plate " + source[-1]

    # Open/create iDOT file
    idot_output_f=open(idot_filename,'a')
    idot_writer = csv.writer(idot_output_f)

    # Write fist line
    now = datetime.now()
    date_str = now.strftime("%m/%d/%Y")
    time_str = now.strftime("%H:%M")

    idot_writer.writerow([protocol_name, software, "<"+user_name+">", date_str, time_str,"","",""])

    for plate in np.sort(idot_df['plateID'].unique()).tolist():

        targetplate_name = plate
        
        # Write second line
        idot_writer.writerow([sourceplate_type, sourceplate_name, "",max_volume_L, target_plate_type, targetplate_name, "",waste_well])

        # Write parameters
        idot_writer.writerow(['DispenseToWaste='+str(dispense_to_waste), 'DispenseToWasteCycles='+ str(dispense_to_waste_cycles), 'DispenseToWasteVolume='+str(dispense_to_waste_volume), 'UseDeionisation='+ str(use_deionisation), 'OptimizationLevel='+optimization_level, 'WasteErrorHandlingLevel='+ waste_error_handling_level, 'SaveLiquids='+ save_liquids,""])

        # Write headers
        idot_writer.writerow(['Source Well', 'Target Well', 'Volume [uL]', 'Liquid Name',"","","",""])

        # Write compounds
        idot_df.loc[(idot_df['sourceID'] == source) & (idot_df['plateID'] == plate)].to_csv(idot_output_f,index=False, header=False, columns=['well_source','well_plaid','CompVol_uL','cmpd_w_stock','X','Y','New Array','New Array'],mode='a')

        # Write dmso Backfill
        idot_df[idot_df.backfill_vol_uL != 0].loc[(idot_df['sourceID'] == source) & (idot_df['plateID'] == plate)].to_csv(idot_output_f,index=False, header=False, columns=['well_backfill','well_plaid','backfill_vol_uL','solvent_w_stock','X','Y','New Array'],mode='a') 
        

    ## Close file before the end
    idot_output_f.close()    

In [76]:
#
# Finally print the protocols where the source plates are spotted
# 

In [77]:
# # Source to source protocols

# target_plate_type = "MWP 96" # Other options are "MWP 96" and "MWP 1536" 


# for i in range(len(sources)-2):

#     protocol_name = "IDOT-" + exp_name + "sourceONsource" + str(i+1)+ '_cmpds' + "_" + solvent
#     idot_filename = "{}/{}.csv".format(OutputDir,protocol_name)
#     sourceplate_name = sources[i]

#     # Open/create iDOT file
#     idot_output_f=open(idot_filename,'a')
#     idot_writer = csv.writer(idot_output_f)

#     # Write fist line
#     now = datetime.now()
#     date_str = now.strftime("%m/%d/%Y")
#     time_str = now.strftime("%H:%M")

#     idot_writer.writerow([protocol_name, software, "<"+user_name+">", date_str, time_str,"","",""])

#     targetplate_name = sources[i+1]

#     # Write second line
#     idot_writer.writerow([sourceplate_type, sourceplate_name, "",max_volume_L, target_plate_type, targetplate_name, "",waste_well])

#     # Write parameters
#     idot_writer.writerow(['DispenseToWaste='+str(dispense_to_waste), 'DispenseToWasteCycles='+ str(dispense_to_waste_cycles), 'DispenseToWasteVolume='+str(dispense_to_waste_volume), 'UseDeionisation='+ str(use_deionisation), 'OptimizationLevel='+optimization_level, 'WasteErrorHandlingLevel='+ waste_error_handling_level, 'SaveLiquids='+ save_liquids, ""])

#     # Write headers
#     idot_writer.writerow(['Source Well', 'Target Well', 'Volume [uL]', 'Liquid Name',"","","",""])

#     # Write compounds
#     idot_df_tosource[idot_df_tosource['sourceID'] == sources[i]].to_csv(idot_output_f,index=False, header=False, columns=['well_source','well','to_source_vol_uL','cmpd_w_stock','X','Y','New Array'],mode='a')

#     # Write Backfills --> quickfix: only add the water backfills
#     idot_df_tosource[(idot_df_tosource['sourceID']  == sources[i]) & (idot_df_tosource['solvent']  == 'water')].to_csv(idot_output_f,index=False, header=False, columns=['well_backfill','well_source','to_source_backfill_vol_uL','solvent_w_stock','X','Y','New Array','New Array'],mode='a')

#     ## Close file before the end
#     idot_output_f.close()

In [78]:
# # DMSO to source protocols

# target_plate_type = "MWP 96" # Other options are "MWP 96" and "MWP 1536" 


# for i in range(len(sources)-2):

#     protocol_name = "IDOT-" + exp_name + "sourceONsource" + str(i+1) + '_backfills' + "_" + solvent
#     idot_filename = "{}/{}.csv".format(OutputDir,protocol_name)
#     sourceplate_name = 'dmso_plate'

#     # Open/create iDOT file
#     idot_output_f=open(idot_filename,'a')
#     idot_writer = csv.writer(idot_output_f)

#     # Write fist line
#     now = datetime.now()
#     date_str = now.strftime("%m/%d/%Y")
#     time_str = now.strftime("%H:%M")

#     idot_writer.writerow([protocol_name, software, "<"+user_name+">", date_str, time_str,"","",""])

#     targetplate_name = sources[i+1]

#     # Write second line
#     idot_writer.writerow([sourceplate_type, sourceplate_name, "",max_volume_L, target_plate_type, targetplate_name, "",waste_well])

#     # Write parameters
#     idot_writer.writerow(['DispenseToWaste='+str(dispense_to_waste), 'DispenseToWasteCycles='+ str(dispense_to_waste_cycles), 'DispenseToWasteVolume='+str(dispense_to_waste_volume), 'UseDeionisation='+ str(use_deionisation), 'OptimizationLevel='+optimization_level, 'WasteErrorHandlingLevel='+ waste_error_handling_level, 'SaveLiquids='+ save_liquids,""])

#     # Write headers
#     idot_writer.writerow(['Source Well', 'Target Well', 'Volume [uL]', 'Liquid Name',"","","",""])

#     # # Write compounds
#     # idot_df_tosource[idot_df_tosource['sourceID'] == sources[i]].to_csv(idot_output_f,index=False, header=False, columns=['well_source','well','to_source_vol_uL','cmpd_w_stock','X','Y','New Array','New Array'],mode='a')

#     # Write DMSO Backfill
#     idot_df_tosource[(idot_df_tosource['sourceID']  == sources[i]) & (idot_df_tosource['solvent']  == 'dmso')].to_csv(idot_output_f,index=False, header=False, columns=['to_source_backfill_well','well_source','to_source_backfill_vol_uL','solvent_w_stock','X','Y','New Array','New Array'],mode='a')

#     ## Close file before the end
#     idot_output_f.close()

## Export idot dataframes

In [79]:
## NO NEED BECAUSE I IMPORTED THEM ALREADY

# Save the idot compound file
name = exp_name + 'idot_cmpds'
idot_df.to_csv("{}/{}_{}.csv".format(SupportDir,name,solvent),index=False, sep=",")


# # Save the idot source file
# name = exp_name + 'idot_source'
# idot_df_tosource.to_csv("{}/{}_{}.csv".format(SupportDir,name, solvent),index=False, sep=",")


# # Save the source layout file
# name = exp_name + 'source'
# df_source.to_csv("{}/{}_{}.csv".format(SupportDir,name, solvent),index=False, sep=",")



### Run afterwards to fix some small stuff

In [80]:
##
## Combine dmso and water cmpds into one file
##

idot_df_dmso = pd.read_csv("{}/{}.csv".format(SupportDir,exp_name + "idot_cmpds_water"),index_col=False)
idot_df_water = pd.read_csv("{}/{}.csv".format(SupportDir, exp_name + "idot_cmpds_dmso"),index_col=False)

idot_df = pd.concat([idot_df_dmso,idot_df_water],axis=0, ignore_index=True)

# Save the idot compound file
name = exp_name + 'idot_cmpds'
idot_df.to_csv("{}/{}.csv".format(SupportDir,name),index=False, sep=",")

## Prepare the checked and combined PLAID files into a metadata file